In [1]:
import numpy as np
import pandas as pd
import os
import ipython_bell
import random

from astropy.io import fits
from astropy.table import Table

In [2]:
data = pd.read_csv('data.csv', sep=',', usecols=[1,2,3,4,5,6])
data.head()

,objectid,RA,dec,grade,photoz,imag
0,139741252,0.59845,-35.12122,2.3,0.51,19.2
1,139823797,0.81825,-33.80120,2.7,0.68,19.3
2,142775105,1.68592,-44.49735,2.0,0.53,18.6
3,142779522,1.87201,-44.57949,3.0,0.52,18.1
4,182452355,2.62678,-43.25413,2.3,0.84,19.9


In [3]:
x = np.zeros((1,3,46,46))
add_info = np.zeros((1, 6))
table_data = Table(names = ['objectid', 'RA', 'dec', 'grade', 'photoz', 'imag'])
path_cut = '/Users/jimenagonzalez/research/DSPL/Searching-double-lenses/real_lenses/cutouts/'
end_name = ['_g.fits', '_i.fits', '_r.fits']

for file in (os.listdir(path_cut)):
    if(file[-6:] == 'g.fits'):
        with fits.open(path_cut + file[:-7] + end_name[0]) as hdul: cutout_g = hdul[0].data
        with fits.open(path_cut + file[:-7] + end_name[1]) as hdul: cutout_i = hdul[0].data
        with fits.open(path_cut + file[:-7] + end_name[2]) as hdul: cutout_r = hdul[0].data
        data_ele = data[data['objectid'] == int(file[:-7])]
        table_data.add_row(data_ele.values[0])
        
        element = np.array([cutout_g, cutout_r, cutout_i])
        x = np.append(x, [element], axis = 0)
x = np.delete(x, 0, axis = 0)

In [4]:
table_data.pprint(5)

  objectid          RA            dec    grade photoz imag
----------- ------------------ --------- ----- ------ ----
324211306.0 42.715709999999994 -16.16883   2.3    0.4 17.7
343290165.0           47.33576 -38.09604   2.7   0.62 18.9
        ...                ...       ...   ...    ...  ...
 87591843.0           32.65344  -53.8646   2.7   0.51 19.7
435151688.0           82.92694 -31.98092   2.0   0.56 18.7
Length = 510 rows


In [5]:
name = 'real_lenses'
primary = fits.PrimaryHDU()
image = fits.ImageHDU(x, name="IMAGE")
table = fits.BinTableHDU(data = table_data)
hdu_list = fits.HDUList([primary, image, table])
hdu_list.writeto(path_cut + name + '.fits', overwrite=True)